# Examining ocean waves with the Whidbey Cable

In [1]:
import h5py
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, detrend
from numpy.fft import fftshift, fft2, fftfreq
import numpy as np
import datetime
import pandas as pd
from dasquakes import *
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from libcomcat.search import search
from libcomcat.dataframes import get_summary_data_frame
import matplotlib.dates as mdates

### Load the DAS Data

In [20]:
def fk_analysis(t0=datetime.datetime(2022, 5, 17, 1, 57, 0),
                draw_figure = True):
    cable = 'whidbey'
    record_length = 10 #minutes


    prefix, network_name, datastore = data_wrangler(cable,record_length,t0)
    data,dates,attrs = open_sintela_file(prefix,
                                         t0,
                                         datastore,
                                         number_of_files=record_length,
                                         verbose=True)
    dx = attrs['SpatialSamplingInterval']
    x_max=data.shape[1] * dx
    date_format = mdates.DateFormatter('%H:%M:%S')
    x_lims = mdates.date2num(dates)
    
    # Entire subsea region:
    x1 = 1225
    x2 = 1600

    # Whidbey half of subsea region:
    # x1 = 1225
    # x2 = 1412

    # Camano half of subsea region:
    # x1 = 1412
    # x2 = 1600

    xmax = 1720

    subsea_data = detrend(data[:,x1:x2])

    ft = fftshift(fft2(subsea_data))
    f = fftshift(fftfreq(subsea_data.shape[0], d=0.01))
    k = fftshift(fftfreq(subsea_data.shape[1], d=attrs['SpatialSamplingInterval']))
    
    
    
    
    '''
    Make a figure
    '''
    if draw_figure == True:
        plt.subplots(1,2,figsize=(15,8))
        das_vmax=1


        low_cut = 0.2
        hi_cut = 2

        b,a = butter(2,(low_cut,hi_cut),'bp',fs=attrs['MaximumFrequency']*2)
        subsea_data_filt = filtfilt(b,a,subsea_data,axis=0)


        ax=plt.subplot(121)
        ax.imshow(subsea_data_filt.T,vmin=-das_vmax,vmax=das_vmax,
                   cmap='seismic',aspect='auto', 
                   extent=[x_lims[0],x_lims[-1],(xmax-x2)*dx,(xmax-x1)*dx])
        ax.xaxis.set_major_formatter(date_format)
        ax.xaxis_date()
        # ax.set_xlim(x_lims[5000],x_lims[15000])
        for i, tick in enumerate(ax.xaxis.get_ticklabels()):
            if i % 2 != 0:
                tick.set_visible(False)

        ax.set_ylabel('Distance (m)',fontsize=18)
        ax.set_xlabel('Time (HH:MM:SS)',fontsize=18)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)



        ax=plt.subplot(122)
        c = ax.imshow(np.log10(np.abs(ft)),
                       aspect='auto',cmap='Greys',
                       vmin=3,vmax=6,
                       extent=[k[0],k[-1],f[0],f[-1]])


        ax.set_ylabel('Frequency (Hz)',fontsize=18)
        ax.set_xlabel('Wavenumber (1/m)',fontsize=18)
        ax.set_ylim([-2.5,2.5])
        # ax.legend()
        # plt.ylim([0])
        ax.set_xlim([-0.04,0.04])
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        # plt.colorbar(c)
        plt.grid()
        plt.show()
        
        
        
    return ft

In [22]:
ft1 = fk_analysis(t0=datetime.datetime(2022, 5, 17, 0, 0, 0),draw_figure=False)

/data/data5/Converted/whidbey_2022-05-17_00-00*.h5
/data/data5/Converted/whidbey_2022-05-17_00-01*.h5
/data/data5/Converted/whidbey_2022-05-17_00-02*.h5
/data/data5/Converted/whidbey_2022-05-17_00-03*.h5
/data/data5/Converted/whidbey_2022-05-17_00-04*.h5
/data/data5/Converted/whidbey_2022-05-17_00-05*.h5
/data/data5/Converted/whidbey_2022-05-17_00-06*.h5
/data/data5/Converted/whidbey_2022-05-17_00-07*.h5
/data/data5/Converted/whidbey_2022-05-17_00-08*.h5
/data/data5/Converted/whidbey_2022-05-17_00-09*.h5


In [23]:
ft1.shape

(59999, 375)

In [24]:
vector1 = ft1.flatten()

In [25]:
vector1.shape

(22499625,)

In [27]:
D = np.zeros((22499625,2))

In [30]:
D[:,0] = np.abs(vector1)